In [1]:
import os
import cv2
import argparse

In [3]:
# construct the argument parser and parser the arguments
parser = argparse.ArgumentParser()
parser.add_argument('-p', '--path', help='what folder to use for HOG description')
args = vars(parser.parse_args())

usage: ipykernel_launcher.py [-h] [-p PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\shubh\AppData\Roaming\jupyter\runtime\kernel-4e13a227-2c4e-4a05-84d2-806caee47747.json


SystemExit: 2

In [ ]:
#fe ex with hog
from skimage.feature import hog
import glob
pl_img = []
images = []
labels = []
for img in glob.glob("extracted/*.jpeg"):
    n= cv2.imread(img,0)
    pl_img.append(n)
    im2arr = np.array(n)
    plt.imshow(n, cmap='gray')
    plt.show()
    
    #fd, hog_image = hog(n,orientations=1,pixels_per_cell=(8,8),cells_per_block=(3,3),block_norm='L2-Hys',visualize=False,transform_sqrt=False,feature_vector=True,multichannel=None)
    H, hog_image = hog(n, orientations=8, pixels_per_cell=(4,4),cells_per_block=(2, 2), visualize=True, multichannel=None, feature_vector=True)
    #plt.axis("off")
    #hog_image = exposure.rescale_intensity(hog_image, out_range=(0, 255)).astype("uint8")
    plt.imshow(hog_image, cmap="gray")
    plt.show()
    np.save('hog_features.npy',H) 
    print(H)
    # update the data and labels
    images.append(hog_image)
    labels.append(img)
    with open("csv/hog_fea.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(H)

In [ ]:
# predict on the test images
print('Evaluating on test images...')
# loop over the test dataset folders
for (i, imagePath) in enumerate(os.listdir(f"test_images/{args['path']}/")):
    image = cv2.imread(f"test_images/{args['path']}/{imagePath}")
    resized_image = cv2.resize(image, (128, 256))
    # get the HOG descriptor for the test image
    (hog_desc, hog_image) = feature.hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys', visualize=True)
    # prediction
    pred = svm_model.predict(hog_desc.reshape(1, -1))[0]
    # convert the HOG image to appropriate data type. We do...
    # ... this instead of rescaling the pixels from 0. to 255.
    hog_image = hog_image.astype('float64')
    # show thw HOG image
    cv2.imshow('HOG Image', hog_image)
    # put the predicted text on the test image
    cv2.putText(image, pred.title(), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
        (0, 255, 0), 2)
    cv2.imshow('Test Image', image)
    cv2.imwrite(f"outputs/{args['path']}_hog_{i}.jpg", hog_image*255.) # multiply by 255. to bring to OpenCV pixel range
    cv2.imwrite(f"outputs/{args['path']}_pred_{i}.jpg", image)
    cv2.waitKey(0)